# SKY130 Inverter Simulation

This notebook demonstrates simulating a CMOS inverter using the open-source SKY130 PDK with NGspice.

**Prerequisites:**
- SKY130 PDK installed (run `./scripts/install_sky130.sh`)
- NGspice installed
- `.spiceinit` copied to home: `cp ~/.ciel/sky130A/libs.tech/ngspice/spinit ~/.spiceinit`

In [1]:
import sys
import os
sys.path.insert(0, '.')

from pade.statement import Analysis, Save, Statement
from pade.backends.ngspice import NgspiceSimulator
from testbenches.inverter_sky130 import InverterSky130TB

# PDK configuration - set PDK_ROOT env var if using custom location
PDK_ROOT = os.environ.get('PDK_ROOT', os.path.expanduser('~/.ciel'))
SKY130_MODELS = f"{PDK_ROOT}/sky130A/libs.tech/ngspice/sky130.lib.spice"

## Create Testbench

The `InverterSky130TB` creates a simple CMOS inverter with:
- 1.8V supply (SKY130 nominal)
- Configurable NMOS/PMOS widths (in um with scale=1e-6)
- Pulse input for transient simulation

In [2]:
# Create testbench: wn=1um, wp=2um, l=150nm
tb = InverterSky130TB(vdd=1.8, wn=1, wp=2, l=0.15)

print(f"Testbench: {tb.cell_name}")
print(f"Subcells: {[c.instance_name for c in tb.get_subcells()]}")

Testbench: tb_inverter_sky130
Subcells: ['Vdd', 'Vin', 'MP', 'MN']


## Setup Simulation

In [3]:
# Simulation statements
statements = [
    Statement(raw=f'.lib "{SKY130_MODELS}" tt'),  # SKY130 typical corner
    Analysis('tran', stop='50n'),
    Save(['inp', 'out', 'vdd']),
]

# Create simulator
sim = NgspiceSimulator(output_dir='sim_data/sim_inverter_sky130')

## View Generated Netlist

In [4]:
# Preview the netlist
netlist = sim.writer.generate_netlist(tb, statements)
print(netlist)

* tb_inverter_sky130 - Generated by PADE

.lib "/Users/fredrief/.ciel/sky130A/libs.tech/ngspice/sky130.lib.spice" tt

* External subcircuit definitions
* SKY130 1.8V PMOS wrapper
* Wraps sky130_fd_pr__pfet_01v8 with simplified interface
* Dimensions in um (with NGspice scale=1e-6)

.subckt pfet_01v8 d g s b
.param w=1 l=0.15 nf=1 mult=1
XM d g s b sky130_fd_pr__pfet_01v8 w={w} l={l} nf={nf} mult={mult}
.ends pfet_01v8
* SKY130 1.8V NMOS wrapper
* Wraps sky130_fd_pr__nfet_01v8 with simplified interface
* Dimensions in um (with NGspice scale=1e-6)

.subckt nfet_01v8 d g s b
.param w=1 l=0.15 nf=1 mult=1
XM d g s b sky130_fd_pr__nfet_01v8 w={w} l={l} nf={nf} mult={mult}
.ends nfet_01v8

* Top-level instances
Vdd vdd 0 dc 1.8
Vin inp 0 pulse(0 1.8 1n 100p 100p 5n 10n)
XMP out inp vdd vdd pfet_01v8 w=2 l=0.15 nf=1 mult=1
XMN out inp 0 0 nfet_01v8 w=1 l=0.15 nf=1 mult=1

.tran 50p 50n
.save v(inp) v(out) v(vdd)
.end


## Run Simulation

In [5]:
# Run transient simulation
raw_path = sim.simulate(tb, statements, 'tran')
print(f"Results saved to: {raw_path}")

[INFO] Netlist written to sim_data/sim_inverter_sky130/tran/tb_inverter_sky130.spice
[INFO] Running: ngspice -b sim_data/sim_inverter_sky130/tran/tb_inverter_sky130.spice -r sim_data/sim_inverter_sky130/tran/raw/output.raw


[INFO] NGspice complete in 27.7s


Results saved to: sim_data/sim_inverter_sky130/tran/raw


## Plot Results

In [6]:
from bokeh.plotting import figure, show, output_notebook
import ltspice

output_notebook()

# Read raw file using ltspice package
raw_file = raw_path / 'output.raw'
l = ltspice.Ltspice(str(raw_file))
l.parse()

# Get time and signals
time = l.get_time()
inp = l.get_data('v(inp)')
out = l.get_data('v(out)')

# Plot
p = figure(title='SKY130 Inverter - Transient Response',
           x_axis_label='Time (ns)', y_axis_label='Voltage (V)',
           width=800, height=400)

p.line(time * 1e9, inp, legend_label='Input (inp)', line_color='blue')
p.line(time * 1e9, out, legend_label='Output (out)', line_color='red')
p.legend.location = 'top_right'

show(p)

Loading BokehJS ...

[Warning] Variable data type is detected as double precision.
